# Exercício - Mini Projeto de Análise de Dados

Vamos fazer um exercício completo de pandas para um miniprojeto de análise de dados.

Esse exercício vai obrigar a gente a usar boa parte dos conhecimento de pandas e até de outros módulos que já aprendemos ao longo do curso.

### O que temos?

Temos os dados de 2019 de uma empresa de prestação de serviços. 

- CadastroFuncionarios
- CadastroClientes
- BaseServiçosPrestados

Obs1: Para ler arquivos csv, temos o read_csv<br>
Obs2: Para ler arquivos xlsx (arquivos em excel normais, que não são padrão csv), temos o read_excel

### O que queremos saber/fazer?

1. Valor Total da Folha Salarial -> Qual foi o gasto total com salários de funcionários pela empresa? <br>
    Sugestão: calcule o salário total de cada funcionário, salário + benefícios + impostos, depois some todos os salários
    
    
2. Qual foi o faturamento da empresa?<br>
    Sugestão: calcule o faturamento total de cada serviço e depois some o faturamento de todos
    
    
3. Qual o % de funcionários que já fechou algum contrato?<br>
    Sugestão: na base de serviços temos o funcionário que fechou cada serviço. Mas nem todos os funcionários que a empresa tem já fecharam algum serviço.<br>
    . Na base de funcionários temos uma lista com todos os funcionários<br>
    . Queremos calcular Qtde_Funcionarios_Fecharam_Serviço / Qtde_Funcionários_Totais<br>
    . Para calcular a qtde de funcionários que fecharam algum serviço, use a base de serviços e conte quantos funcionários tem ali. Mas lembre-se, cada funcionário só pode ser contado uma única vez.<br><br>
    Dica: se você aplicar o método .unique() em uma variável que é apenas 1 coluna de um dataframe, ele vai excluir todos os valores duplicados daquela coluna.<br>
    Ex: unicos_colunaA = dataframe['colunaA'].unique() te dá como resposta uma lista com todos os itens da colunaA aparecendo uma única vez. Todos os valores repetidos da colunaA são excluidos da variável unicos_colunaA 
    
    
4. Calcule o total de contratos que cada área da empresa já fechou


5. Calcule o total de funcionários por área


6. Qual o ticket médio mensal (faturamento médio mensal) dos contratos?<br>
    Dica: .mean() calcula a média -> exemplo: media_colunaA = dataframe['colunaA'].mean()

Obs: Lembrando as opções mais usuais de encoding:<br>
encoding='latin1', encoding='ISO-8859-1', encoding='utf-8' ou então encoding='cp1252'

Observação Importante: Se o seu código der um erro na hora de importar os arquivos:<br>
- CadastroClientes.csv
- CadastroFuncionarios.csv

Use separador ";" (ponto e vírgula) para resolver

### Importação de Módulos e Arquivos

In [61]:
import pandas as pd

funcionarios_df = pd.read_csv(r'CadastroFuncionarios.csv', sep=';', decimal=',')
clientes_df = pd.read_csv(r'CadastroClientes.csv')
servicosPrestados_df = pd.read_excel(r'BaseServiçosPrestados.xlsx')


### 1 - Folha Salarial

In [63]:
# Minha resolução para o primeiro problema

# limpar base de dados, retirando colunas que não serão usadas

#funcionarios_df = funcionarios_df.drop(['Estado Civil', 'Cargo'], axis=1)

#funcionarios_df.info()

vlrTotalFolha = funcionarios_df.groupby('Nome Completo').sum()

vlrTotalFolha = vlrTotalFolha['Salario Base'].sum() + vlrTotalFolha['Impostos'].sum() + vlrTotalFolha['Beneficios'].sum() + vlrTotalFolha['VT'].sum() + vlrTotalFolha['VR'].sum()

vlrTotalFolha = 'R${:_.2f}'.format(vlrTotalFolha)
vlrTotalFolha = vlrTotalFolha.replace('.', ',').replace('_', '.')
print("O valor total da folha foi de: {}".format(vlrTotalFolha))

O valor total da folha foi de: R$2.717.493,22


In [64]:
#Resolução do professor da Aula
funcionarios_df['Salario Total'] = funcionarios_df['Salario Base'] + funcionarios_df['Impostos'] + funcionarios_df['Beneficios'] + funcionarios_df['VT'] + funcionarios_df['VR']
print('Total da Folha Salarial Mensal é de R${:,}'.format(funcionarios_df['Salario Total'].sum()))

Total da Folha Salarial Mensal é de R$2,717,493.22


### 2 - Faturamento da Empresa

### 3 - % Funcionários que Fecharam Contrato

### 4 - Qtde de Contratos por Área

### 5 - Total de Funcionários por área

### 6 - Ticket médio mensal